In [2]:
import requests
from bs4 import BeautifulSoup
import json
from time import time

from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import ProcessPoolExecutor
from concurrent.futures import as_completed

/Users/Welcome/Library/Python/3.9/lib/python/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn(


In [3]:
#utils.py part
TYPE_CONFIG = {
    "사회": "society",
    "정치": "politics",
    "경제": "economic",
    "국제": "foreign",
    "문화": "culture",
    "연예": "entertain",
    "스포츠": "sports",
    "IT": "digital",
    "칼럼": "editorial",
    "보도자료": "press",
}

In [40]:
#**kwars 이거써서 코드 깔끔하게
headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'}

def has_next_btn(html) -> bool:
    next_btn = html.select("div.box_etc > div.paging_news > span.inner_paging > a.btn_page.btn_next")
    if len(next_btn) == 0:
        return False
    else:
        return True
    
def makeURL_naver(search = ""):
    print("makeURL_naver")
    
def set_params() -> dict:
    print({"params":"params"})

def makeURL_daum(search = "", period="", start_date="", end_date="") -> str:
    url = "https://search.daum.net/search?w=news&DA=STC&enc=utf8"
    period_url = ""
    search_url = ""
    
    if period != "": #[TODO] 올바른 period가 안들어왔을때 에러 핸들링
        period_url = "&period=%s" % (period)
        
        if period == "u": #기간 조정
            period_url = period_url + "&sd=%s&ed=%s" % (start_date, end_date)
    
    if search !="":
        search_url = "&q=%s" % (search)
    return url + period_url + search_url

def return_select_func(html, request_type) -> list:
    if request_type == "search":
        return html.select("div.cont_divider > ul.list_news > li div.wrap_cont > a")
    elif request_type == "normal":
        return html.select("div.box_etc > ul.list_news2.list_allnews > li div.cont_thumb > strong.tit_thumb > a")

def extract_title(url, request_type) -> bool:
    try:
        original_html = requests.get(url)
        html = BeautifulSoup(original_html.text, "html.parser")        
        title_list = return_select_func(html, request_type)
        
        print([title.get_text() for title in title_list])
        if len(title_list) == 0 or not has_next_btn(html):
            return False
    except:
        return False

    return True

def newsScrape(search):
    URL = makeURL_daum(search)
    for page_number in range(1, 11):
        try:
            original_html = requests.get(URL + "&page=" + str(page_number), headers = headers)
            html = BeautifulSoup(original_html.text, "html.parser")
            title_list = html.select("div.cont_divider > ul.list_news > li div.wrap_cont > a")
            if len(title_list) == 0:
                break
        except:
            break


        
def newsType(newsType="", date=""):
    THREAD_COUNT = 10
    URL = "https://news.daum.net/breakingnews/"
    if newsType != "":
        URL = f"{URL}{TYPE_CONFIG[newsType]}"
    
    if date != "": 
        URL = f"{URL}?regDate={date}" 
        
    page_num = 1
    with ThreadPoolExecutor(THREAD_COUNT) as executor:
        while(True):
            url = f"{URL}&page={str(page_num)}"
            future = executor.submit(extract_title, url, "normal")
            page_num = page_num + 1
            break
            if not future.result():
                break


if __name__ == '__main__':        
    start = time()
    newsType("경제", "20220720")
    #newsScrape("우영우")
    print("time : ", time() - start)
    
    
#[TODO] TYPE_CONFIG에 들어있지 않는 타입 들어왔을때 에러 핸들링
#[TODO] 올바른 date가 아닐때 에러 핸들링

['대우조선 하청노사 협상 불발..손해배상 이견 극복 실패', "'9부 능선' 넘은 대우조선해양 하청 노사..21일 교섭 재개", '코인·주식 다음은 부동산?..美 모기지 신청 22년래 최저', '7개 수입 생필품에 0% 할당관세..수입 쇠고기 첫 적용', "대우조선 하청 노사협상 불발.. 쟁점은 '손해배상 소송'", '베이커휴즈, 2분기 실적 월가 기대치 못미쳐 13% 급락', '美 6월 주택판매 감소에 2분기 GDP 전망치 하향', '대우조선 협력업체 노사 합의 불발..내일 오전 10시 재협상(2보)', "[마켓인]롯데지주, 공모채 수요예측 연타석 '흥행'", '뉴욕증시, 실적 발표 주시하며 혼조세로 출발', '18개국 장관 "공급망 위기 극복 공조"..공급망 탈중국 본격화 하나', '[단독]해시드, 테라·루나 폭락 전 1300억 매도..모럴헤저드 논란', "동행복권, 복권판매인 및 소비자와 함께한 '따뜻한 동행'", '"공급망 위기에 긴밀히 공조" 외친 17개국 장관들..中만 없었다', '美 뉴욕 증시, 넷플릭스 실적 선방에 혼조 출발']
time :  0.3543109893798828
